# COVID-19 forecast in French departments

#### <br> Visualize a forecast (linear growth by department) of the evolution of the COVID-19 hospitalizations in French departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import requests
import zipfile
import io
from datetime import timedelta, date
import numpy as np
import pandas as pd
import geopandas as gpd
from fbprophet import Prophet
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import contextily as ctx
from PIL import Image
import warnings
import logging

logging.getLogger("fbprophet").setLevel(logging.WARNING)
warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

#### <br> COVID and geographical data are open data from the French open data portal data.gouv.fr

In [3]:
url_dep = "http://osm13.openstreetmap.fr/~cquest/openfla/export/departements-20140306-5m-shp.zip"
url_covid = (
    "https://www.data.gouv.fr/fr/datasets/r/b94ba7af-c0d6-4055-a883-61160e412115"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
nb_days = 9  # forecast horizon (days)
figsize = (15, 15)
tile_zoom = 7
frame_duration = 1000  # in ms

#### <br> Load French departements data into a GeoPandas GeoSeries

In [4]:
local_path = "tmp/"
r = requests.get(url_dep)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path=local_path)
filenames = [
    y
    for y in sorted(z.namelist())
    for ending in ["dbf", "prj", "shp", "shx"]
    if y.endswith(ending)
]
dbf, prj, shp, shx = [filename for filename in filenames]
fr = gpd.read_file(local_path + shp)  #  + encoding="utf-8" if needed
met = fr.query("code_insee not in @filter_dep")
met.set_index("code_insee", inplace=True)
met = met["geometry"]

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


#### <br> Load the map tile with contextily

In [5]:
w, s, e, n = met.total_bounds
bck, ext = ctx.bounds2img(w, s, e, n, zoom=tile_zoom, ll=True)

#### <br> Plot function to save image at a given date (title)

In [6]:
def save_img(df, title, img_name, vmin, vmax):
    gdf = gpd.GeoDataFrame(df, crs={"init": "epsg:4326"})
    gdf_3857 = gdf.to_crs(epsg=3857)  # web mercator
    f, ax = plt.subplots(figsize=figsize)
    ax.imshow(
        bck, extent=ext, interpolation="sinc", aspect="equal"
    )  # load background map
    divider = make_axes_locatable(ax)
    cax = divider.append_axes(
        "right", size="5%", pad=0.1
    )  # GeoPandas trick to adjust the legend bar
    gdf_3857.plot(
        column="hosp",  # Number of people currently hospitalized
        ax=ax,
        cax=cax,
        alpha=0.75,
        edgecolor="k",
        legend=True,
        cmap=matplotlib.cm.get_cmap("magma_r"),
        vmin=vmin,
        vmax=vmax,
    )

    ax.set_axis_off()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title(title, fontsize=25)
    plt.savefig(img_name, bbox_inches="tight")  # pad_inches=-0.1 to remove border
    plt.close(f)

#### <br> Load COVID data into a pandas DataFrame

In [7]:
cov = pd.read_csv(url_covid, sep=";", index_col=2, parse_dates=True,)
cov = cov.query("sexe == 0")  # sum of male/female
cov = cov.query("dep not in @filter_dep")
cov.dropna(inplace=True)
cov.drop(columns=["sexe"], inplace=True)
cov.sort_index(inplace=True)
cov.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


#### <br> Forecast with Prophet each department hosp time serie evolution

In [8]:
latest_historical_day = cov.index.max()
print("Forecast with data before: ", latest_historical_day)
dep_df_list = []
for dep in cov.dep.unique():
    sdf = cov[cov["dep"] == dep]
    sdf = sdf[["hosp"]]
    sdf = sdf.reset_index()
    sdf.rename(columns={"jour": "ds", "hosp": "y"}, inplace=True)
    m = Prophet(changepoint_prior_scale=0.05)
    m.fit(sdf)
    future = m.make_future_dataframe(periods=nb_days)
    after = m.predict(future)
    after = after[["ds", "trend"]]
    after.rename(columns={"ds": "jour", "trend": "hosp"}, inplace=True)
    after = after[after["jour"] > latest_historical_day]
    after.set_index("jour", inplace=True)
    after["hosp"] = after["hosp"].apply(np.int64)
    after["dep"] = dep
    dep_df_list.append(after)

future_cov = pd.concat(dep_df_list)
future_cov.head()

Forecast with data before:  2020-03-25 00:00:00


,hosp,dep
jour,,
2020-03-26,28,01
2020-03-27,32,01
2020-03-28,36,01
2020-03-29,40,01
2020-03-30,44,01


#### <br> Add geometry data to forecast COVID DataFrame

In [9]:
future_cov["geometry"] = future_cov["dep"].map(met)

#### <br> Parse days and save one image for each day

In [10]:
def daterange(date1, date2):
    for n in range(int((date2 - date1).days) + 1):
        yield date1 + timedelta(n)

In [11]:
vmax = cov.hosp.max()
for i, dt in enumerate(daterange(future_cov.index.min(), future_cov.index.max())):
    title = dt.strftime("%d-%b-%Y")
    df = future_cov.query("jour == @dt")
    df = df.drop_duplicates(subset=["dep"], keep="first")
    img_name = "img/" + str(i) + ".png"
    save_img(df, title, img_name, 0, vmax)

#### <br> Compile images in animated gif

In [12]:
frames = []
for i, dt in enumerate(daterange(future_cov.index.min(), future_cov.index.max())):
    name = "img/" + str(i) + ".png"
    frames.append(Image.open(name))

frames[0].save(
    "forecastcovid.gif",
    format="GIF",
    append_images=frames[1:],
    save_all=True,
    duration=frame_duration,
    loop=0,
)

from IPython.display import HTML

HTML("<img src='forecastcovid.gif'>")